```{contents}
```
## **Session Store in LangGraph**

A **Session Store** in LangGraph is the **persistent storage layer** that maintains execution context across time, requests, and system restarts.
It allows LangGraph to support **long-running workflows, multi-turn agents, resumable execution, human-in-the-loop, and production reliability**.

---

### **1. Why Session Store Exists**

LLM workflows are **not single function calls**.
They span **multiple user interactions**, background tasks, tool calls, and human approvals.

Without a session store:

* Conversations cannot continue across requests
* Execution state is lost on crashes
* Long-running workflows cannot be resumed
* Human review becomes impossible

LangGraph solves this with a **session store**.

---

### **2. What a Session Contains**

A **session** represents one continuous logical execution.

| Component          | Purpose                           |
| ------------------ | --------------------------------- |
| Session ID         | Unique workflow identity          |
| State Snapshot     | Latest global state               |
| Execution Position | Which node is active              |
| Graph Version      | Which graph definition is running |
| Checkpoints        | Historical execution states       |
| Metadata           | User, timestamps, permissions     |
| Thread ID          | Conversation or task identity     |

---

### **3. Relationship Between Session, Thread, and Run**

| Concept | Meaning                        |
| ------- | ------------------------------ |
| Thread  | Conversation / task identity   |
| Session | Persistent execution container |
| Run     | One invocation of the graph    |

One **thread** can have multiple **runs**, all tied to one **session**.

---

### **4. How Session Store Works Internally**

```
User Request
   ↓
LangGraph Runtime
   ↓
Load Session from Store
   ↓
Resume Execution at Saved Node
   ↓
Execute Next Step
   ↓
Persist Updated State + Checkpoint
   ↓
Return Output
```

---

### **5. Implementation Example**

```python
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()

graph = builder.compile(checkpointer=checkpointer)

graph.invoke(
    {"input": "Start analysis"},
    config={"configurable": {"thread_id": "user-42"}}
)
```

Here:

* `thread_id` identifies the **session**
* `MemorySaver` is the **session store**
* Every step is **persisted automatically**

---

### **6. Production Session Stores**

| Store         | Use Case                 |
| ------------- | ------------------------ |
| In-memory     | Development only         |
| Redis         | High-throughput sessions |
| PostgreSQL    | Durable persistence      |
| DynamoDB      | Serverless scale         |
| Cloud storage | Long-term archives       |

---

### **7. What Gets Persisted**

| Layer        | Stored                |
| ------------ | --------------------- |
| State        | Full typed state      |
| Execution    | Current node, edges   |
| History      | All checkpoints       |
| Metadata     | Timestamps, ownership |
| Human inputs | Review & corrections  |

---

### **8. Enabling Resume & Replay**

```python
graph.invoke(None, config={"configurable": {"thread_id": "user-42"}})
```

Execution resumes from the **last saved step**.

---

### **9. Why Session Store is Critical for Production**

| Feature               | Requires Session Store |
| --------------------- | ---------------------- |
| Long conversations    | Yes                    |
| Human-in-the-loop     | Yes                    |
| Crash recovery        | Yes                    |
| Distributed execution | Yes                    |
| Audit & compliance    | Yes                    |
| Autonomous agents     | Yes                    |

---

### **10. Mental Model**

A session store makes LangGraph behave like:

> **A database-backed state machine with memory**

This transforms LLM workflows into **reliable software systems** rather than fragile scripts.

---

### **11. Common Failure Without Session Store**

| Without       | Result              |
| ------------- | ------------------- |
| Process crash | All progress lost   |
| User leaves   | Conversation resets |
| Human review  | Cannot resume       |
| Scaling       | State corruption    |

---

### **12. Advanced Production Controls**

* Encryption at rest
* State versioning
* Snapshot pruning
* Access control
* Multi-tenant isolation
* Session expiry policies


### Demonstration

In [1]:
# -------- LangGraph Session Store Demo (Single Cell) --------

from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# 1. Define state schema
class State(TypedDict):
    counter: int

# 2. Define nodes
def increment(state: State):
    print(f"Current counter: {state['counter']}")
    return {"counter": state["counter"] + 1}

def check_done(state: State):
    if state["counter"] >= 3:
        print("Stopping condition reached")
        return END
    return "increment"

# 3. Build graph
builder = StateGraph(State)
builder.add_node("increment", increment)
builder.set_entry_point("increment")
builder.add_conditional_edges("increment", check_done, {
    "increment": "increment",
    END: END
})

# 4. Attach session store
session_store = MemorySaver()
graph = builder.compile(checkpointer=session_store)

# 5. First execution (creates session + checkpoints)
print("\n--- First Run ---")
graph.invoke({"counter": 0}, config={"configurable": {"thread_id": "demo-session"}})

# 6. Simulate crash / new process: Resume same session
print("\n--- Resume Run ---")
graph.invoke(None, config={"configurable": {"thread_id": "demo-session"}})



--- First Run ---
Current counter: 0
Current counter: 1
Current counter: 2
Stopping condition reached

--- Resume Run ---


{'counter': 3}